In [32]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

d = ctrl.Antecedent(np.arange(0, 51, 1), 'zadany_zasieg')
alpha = ctrl.Antecedent(np.arange(10, 81, 1), 'kat_rzutu')
k = ctrl.Antecedent(np.arange(0, 0.5, 0.1), 'wspolczynnik_oporu')
v0 = ctrl.Consequent(np.arange(0, 91, 1), 'predkosc_poczatkowa')

d['s'] = fuzz.trimf(d.universe, [0, 0, 25])
d['m'] = fuzz.trimf(d.universe, [20, 25, 40])
d['l'] = fuzz.trimf(d.universe, [35, 50, 50])

alpha['s'] = fuzz.trimf(alpha.universe, [20, 20, 40])
alpha['m'] = fuzz.trimf(alpha.universe, [35, 45, 55])
alpha['l'] = fuzz.trimf(alpha.universe, [40, 70, 70])

k['s'] = fuzz.trimf(k.universe, [0, 0, 0.2])
k['m'] = fuzz.trimf(k.universe, [0.15, 0.25, 0.35])
k['l'] = fuzz.trimf(k.universe, [0.3, 0.5, 0.5])

v0['s'] = fuzz.trimf(v0.universe, [5, 5, 20])
v0['m'] = fuzz.trimf(v0.universe, [15, 25, 35])
v0['l'] = fuzz.trimf(v0.universe, [30, 50, 50])

# d - zasieg rzutu, alpha - kat rzutu, k - wsp oporu powietrza
rules = [
    #
    ctrl.Rule(alpha['m'] & d['s'], v0['s']),
    ctrl.Rule(alpha['m'] & (d['l'] | d['m']), v0['m']),
    ctrl.Rule(k['l'], v0['m']),
    ctrl.Rule(alpha['s'] | alpha['l'], v0['m']),
    ctrl.Rule(k['l'] & (d['l'] | d['m']), v0['l']),
    #
    ctrl.Rule(d['s'] & alpha['s'] & k['s'], v0['s']),
    ctrl.Rule(d['s'] & alpha['m'] & k['s'], v0['s']),
    ctrl.Rule(d['s'] & alpha['l'] & k['s'], v0['s']),
    #
    ctrl.Rule(d['s'] & alpha['s'] & k['m'], v0['s']),
    ctrl.Rule(d['s'] & alpha['m'] & k['m'], v0['s']),
    ctrl.Rule(d['s'] & alpha['l'] & k['m'], v0['s']),
    #
    ctrl.Rule(d['s'] & alpha['s'] & k['l'], v0['m']),
    ctrl.Rule(d['s'] & alpha['m'] & k['l'], v0['s']),
    ctrl.Rule(d['s'] & alpha['l'] & k['l'], v0['m']),
    #
    ctrl.Rule(d['m'] & alpha['s'] & k['s'], v0['m']),
    ctrl.Rule(d['m'] & alpha['m'] & k['s'], v0['s']),
    ctrl.Rule(d['m'] & alpha['l'] & k['s'], v0['m']),
    #
    ctrl.Rule(d['m'] & alpha['s'] & k['m'], v0['m']),
    ctrl.Rule(d['m'] & alpha['m'] & k['m'], v0['s']),
    ctrl.Rule(d['m'] & alpha['l'] & k['m'], v0['m']),
    #
    ctrl.Rule(d['m'] & alpha['s'] & k['l'], v0['m']),
    ctrl.Rule(d['m'] & alpha['m'] & k['l'], v0['m']),
    ctrl.Rule(d['m'] & alpha['l'] & k['l'], v0['m']),
    #
    ctrl.Rule(d['l'] & alpha['s'] & k['s'], v0['l']),
    ctrl.Rule(d['l'] & alpha['m'] & k['s'], v0['m']),
    ctrl.Rule(d['l'] & alpha['l'] & k['s'], v0['l']),
    #
    ctrl.Rule(d['l'] & alpha['s'] & k['m'], v0['l']),
    ctrl.Rule(d['l'] & alpha['m'] & k['m'], v0['m']),
    ctrl.Rule(d['l'] & alpha['l'] & k['m'], v0['l']),
    #
    ctrl.Rule(d['l'] & alpha['s'] & k['l'], v0['l']),
    ctrl.Rule(d['l'] & alpha['m'] & k['l'], v0['m']),
    ctrl.Rule(d['l'] & alpha['l'] & k['l'], v0['l'])
]

model = ctrl.ControlSystem(rules)
v0est = ctrl.ControlSystemSimulation(model)

def predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc):
    v0est.input['zadany_zasieg'] = zadany_zasieg_wartosc
    v0est.input['kat_rzutu'] = kat_rzutu_wartosc
    v0est.input['wspolczynnik_oporu'] = wspolczynnik_oporu_wartosc
    v0est.compute()
    return v0est.output['predkosc_poczatkowa']

zadany_zasieg_wartosc = 30
kat_rzutu_wartosc = 45
wspolczynnik_oporu_wartosc = 0.1

predkosc_pocz = predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc)
print("Szacowana prędkość początkowa dla zasięgu rzutu", zadany_zasieg_wartosc, "to:", predkosc_pocz)


Szacowana prędkość początkowa dla zasięgu rzutu 30 to: 19.1969111969112


In [31]:
import math

def zasieg_rzutu(V0, alpha, k):
    g = 9.81
    xmax = (V0**2 * math.sin(math.radians(2 * alpha))) / (g + k * V0)
    return xmax

# Metoda zlotego podzialu
def znajdz_poczatkowa_predkosc(d, alfa, k, epsilon=0.1):
    a = 0.1
    b = 100
    
    while True:
        V0 = (a + b) / 2
        xmax = zasieg_rzutu(V0, alfa, k)

        if abs(xmax - d) < epsilon:
            return V0
        elif xmax < d:
            a = V0
        else:
            b = V0

zasieg = 30
kat = 45
opor_powietrza = 0.1
V0 = znajdz_poczatkowa_predkosc(zasieg, kat, opor_powietrza)
print("Prędkość początkowa rzutu ukośnego:", V0, "m/s")


Prędkość początkowa rzutu ukośnego: 18.733691406250003 m/s
